In [1]:
import numpy as np

In [49]:
class HMM:
    def __init__(self, stateSet, observationSet, transMatrix, obsMatrix, initDist ):
        
        self.stateSet = stateSet
        self.observationSet = observationSet
        
        self.nStates = len(stateSet)
        self.nObs = len(observationSet)    
        
        self.transMatrix = np.matrix(transMatrix)
        self.obsMatrix = np.matrix(obsMatrix)
        self.initDist = np.matrix(initDist)
        
        if(self.transMatrix.shape != (self.nStates, self.nStates)):
            raise ValueError("La matriz de transición debe ser cuadrada de orden número de estados")
            
        if(self.obsMatrix.shape != (self.nStates, self.nObs)):
            raise ValueError("La matriz de emisión debe ser de orden número de estados por número de observaciones")
            
        if(self.initDist.shape != (1,self.nStates)):
            raise ValueError("La distribución inicial debe ser un vector de dimensión el número de estados")

        checkStochastic = lambda M: all(np.sum(row)==1.0 for row in M)
        
        if(not checkStochastic(self.transMatrix) or not checkStochastic(self.obsMatrix) or not checkStochastic(self.initDist)):
            raise ValueError("Las filas deben sumar 1")
        
    # Parameters:
    # obsIndSeq: secuencia de observaciones indexado    
    def computeAlpha(self, obsIndSeq):
        
        #Calcular α_0(i)
        Alpha=[ [self.initDist[i]*self.nObs[i][obsIndSeq[0]] for i in range(self.nStates) ] ]
        C = [ np.sum(Alpha[0]) ]
        
        
        
        
        
        
    

In [42]:
example = HMM(2,3, np.matrix(np.matrix([[0.1,0.9],[0.2,0.8]])), np.matrix(np.matrix([[0.1,0.6,0.3],[0.2,0.8,0.0]])), np.matrix([0.5,0.5]) )

In [22]:
np.matrix(np.array([0.5,0.5])).shape

(1, 2)

In [45]:
A=np.matrix([[0.1,0.9],[0.2,0.8]])
for a in A:
    print(np.sum(a)==1.0)

True
True


In [67]:
B=np.matrix([1,2])
B=np.vstack([B,np.matrix([2,2])])
B

matrix([[1, 2],
        [2, 2]])